# Useful standard library modules
(pathlib, shutil, sys, os, subprocess, zipfile, etc.)

These packages are part of the standard python library and provide very useful functionality for working with your operating system and files.  This notebook will provide explore these packages and demonstrate some of their functionality.  Online documentation is at https://docs.python.org/3/library/.


#### Topics covered:
* **``pathlib``**:
    * listing files
    * creating, moving and deleting files
    * absolute vs relative paths
    * useful path object attributes
* **``shutil``**: 
    * copying, moving and deleting files AND folders
* **``sys``**: 
    * python and platform information
    * command line arguments
    * modifying the python path to import code from other locations
* **``os``**:
    * changing the working directory
    * recursive iteration through folder structures
    * accessing environmental variables
* **``subprocess``**: 
    * running system commands and checking the results
* **``zipfile``**:
    * creating and extracting from zip archives

In [7]:
import os
from pathlib import Path
import shutil
import subprocess
import sys
import zipfile

## ``pathlib`` — Object-oriented filesystem paths
Pathlib provides convenient "pathlike" objects for working with file paths across platforms (meaning paths or operations done with pathlib work the same on Windows or POSIX systems (Linux, OSX, etc)). The main entry point for users is the ``Path()`` class.

further reading:  
https://treyhunner.com/2018/12/why-you-should-be-using-pathlib/  
https://docs.python.org/3/library/pathlib.html

#### Make a ``Path()`` object for the current folder

In [10]:
cwd = Path('.')
cwd

PosixPath('.')

### Listing files

In [11]:
for f in cwd.iterdir():
    print(f)

.DS_Store
00_python_basics_review.ipynb
test.py
numpy.ipynb
03_sys-os-shutil-pathlib.ipynb
.ipynb_checkpoints
data


#### List just the notebooks using the ``.glob()`` method

In [12]:
for nb in cwd.glob('*.ipynb'):
    print(nb)

00_python_basics_review.ipynb
numpy.ipynb
03_sys-os-shutil-pathlib.ipynb


#### Note: ``.glob()`` works across folders too
List all notebooks for both class components

In [16]:
for nb in cwd.glob('../*/*.ipynb'):
    print(nb)

../part0_python_intro/00_python_basics_review.ipynb
../part0_python_intro/numpy.ipynb
../part0_python_intro/03_sys-os-shutil-pathlib.ipynb


#### But ``glob`` results aren't sorted alphabetically!
(and the sorting is platform-dependent)

https://arstechnica.com/information-technology/2019/10/chemists-discover-cross-platform-python-scripts-not-so-cross-platform/?comments=1&post=38113333

we can easily sort them by casting the results to a list

In [53]:
sorted(list(cwd.glob('../*/*.ipynb')))

[PosixPath('../part0_python_intro/00_python_basics_review.ipynb'),
 PosixPath('../part0_python_intro/03_sys-os-shutil-pathlib.ipynb'),
 PosixPath('../part0_python_intro/numpy.ipynb')]

**Note:** There is also a glob module in the standard python library that works directly with string paths

In [120]:
import glob
sorted(list(glob.glob('../*/*.ipynb')))

['../part0_python_intro/00_python_basics_review.ipynb',
 '../part0_python_intro/03_sys-os-shutil-pathlib.ipynb',
 '../part0_python_intro/numpy.ipynb']

#### List just the subfolders

In [17]:
[f for f in cwd.iterdir() if f.is_dir()]

[PosixPath('.ipynb_checkpoints'), PosixPath('data')]

#### Create a new path for the data subfolder

In [18]:
data_path = cwd / 'data'
data_path

PosixPath('data')

#### or an individual file

In [20]:
f = cwd / '00_python_basics_review.ipynb'
f

PosixPath('00_python_basics_review.ipynb')

#### check if it exists, or if it's a directory

In [22]:
f.exists(), f.is_dir()

(True, False)

### Creating files and folders

#### make a new subdirectory

In [23]:
new_folder = cwd / 'more_files'
new_folder

PosixPath('more_files')

In [24]:
new_folder.exists()

False

In [26]:
new_folder.mkdir(); new_folder.exists()

FileExistsError: [Errno 17] File exists: 'more_files'

Note that if you try to run the above cell twice, you'll get an error that the folder already exists
``exist_ok=True`` supresses these errors.

In [28]:
new_folder.mkdir(exist_ok=True)

#### make a new subfolder within a new subfolder
The ``parents=True`` argument allows for making subfolders within new subfolders

In [29]:
(new_folder / 'subfolder').mkdir(exist_ok=True, parents=True)

### absolute vs. relative pathing

Get the absolute location of the current working directory

In [43]:
abs_cwd = Path.cwd()
abs_cwd

PosixPath('/Users/aleaf/Documents/Python_course/python-for-hydrology/notebooks/part0_python_intro')

Go up two levels to the course repository

In [44]:
class_root = (abs_cwd / '../../')
class_root

PosixPath('/Users/aleaf/Documents/Python_course/python-for-hydrology/notebooks/part0_python_intro/../..')

Simplify or resolve the path

In [48]:
class_root = class_root.resolve()
class_root

PosixPath('/Users/aleaf/Documents/Python_course/python-for-hydrology')

Get the cwd relative to the course repository

In [49]:
abs_cwd.relative_to(class_root)

PosixPath('notebooks/part0_python_intro')

check if this is an absolute or relative path

In [50]:
abs_cwd.relative_to(class_root).is_absolute()

False

In [51]:
abs_cwd.is_absolute()

True

### useful attributes

In [52]:
abs_cwd.parent

PosixPath('/Users/aleaf/Documents/Python_course/python-for-hydrology/notebooks')

In [60]:
abs_cwd.parent.parent

PosixPath('/Users/aleaf/Documents/Python_course/python-for-hydrology')

In [56]:
f.name

'00_python_basics_review.ipynb'

In [57]:
f.suffix

'.ipynb'

In [58]:
f.with_suffix('.junk')

PosixPath('00_python_basics_review.junk')

In [59]:
f.stem

'00_python_basics_review'

### Moving and deleting files

Make a file

In [61]:
fname = Path('new_file.txt')
with open(fname, 'w') as dest:
    dest.write("A new text file.")

In [62]:
fname.exists()

True

Delete the file

In [67]:
fname.unlink()

In [68]:
fname.exists()

False

#### Delete the empty folder we made above
Note: this only works for empty directories (use ``shutil.rmtree()`` very carefully for removing folders and all contents within)

In [66]:
Path('more_files/subfolder/').rmdir()

## ``shutil`` — High-level file operations
module for copying, moving, and deleting files and directories.

https://docs.python.org/3/library/shutil.html

The functions from shutil that you may find useful are:

    shutil.copy()
    shutil.copy2()  # this preserves most metadata (i.e. dates); unlike copy()
    shutil.copytree()
    shutil.move()
    shutil.rmtree()  #obviously, you need to be careful with this one!
    
Give these guys a shot and see what they do.  Remember, you can always get help by typing:

    help(shutil.copy)


In [121]:
#try them here.  Be careful!

## ``sys`` — System-specific parameters and functions

### Getting information about python and the os
where python is installed

In [8]:
print(sys.prefix)

/Users/aleaf/anaconda3/envs/pyclass


In [9]:
print(sys.version_info)

sys.version_info(major=3, minor=7, micro=3, releaselevel='final', serial=0)


In [10]:
sys.platform

'darwin'

### Adding command line arguments to a script
Here the command line arguments reflect that we're running a Juptyer Notebook. 

In a python script, command line arguments are listed after the first item in the list.

In [5]:
sys.argv

['/Users/aleaf/opt/anaconda3/envs/gis/lib/python3.10/site-packages/ipykernel_launcher.py',
 '-f',
 '/Users/aleaf/Library/Jupyter/runtime/kernel-b22b3c3e-1d83-473d-83a6-1a4876a6b1f5.json']

### Exercise: Make a script with a command line argument using sys.argv

1) Using a text editor such as VSCode, make a new ``*.py`` file with the following contents:

```python
import sys

if len(sys.argv) > 1:
    for argument in sys.argv[1:]:
        print(argument)
else:
    print("usage is: python <script name>.py argument")
    quit()
```

2) Try running the script at the command line

### modifying the python path

If you haven't seen `sys.path` already mentioned in a python script, you will soon.  `sys.path` is a list of directories.  This path list is used by python to search for python modules and packages.  If for some reason, you want to use a python package or  module that is not installed in the main python folder, you can add the directory containing your module to sys.path.

Any packages installed by linking the source code in place (i.e. ``pip install -e .`` will also show up here.

In [141]:
for pth in sys.path:
    print(pth)

/Users/aleaf/Documents/Python_course/python-for-hydrology/notebooks/part0_python_intro
/Users/aleaf/opt/anaconda3/envs/gis/lib/python310.zip
/Users/aleaf/opt/anaconda3/envs/gis/lib/python3.10
/Users/aleaf/opt/anaconda3/envs/gis/lib/python3.10/lib-dynload

/Users/aleaf/opt/anaconda3/envs/gis/lib/python3.10/site-packages
/Users/aleaf/opt/anaconda3/envs/gis/lib/python3.10/site-packages/GDAL-3.5.1-py3.10-macosx-10.9-x86_64.egg
/Users/aleaf/opt/anaconda3/envs/gis/lib/python3.10/site-packages/PyQt5_sip-12.11.0-py3.10-macosx-10.9-x86_64.egg
/Users/aleaf/Documents/GitHub/sfrmaker
/Users/aleaf/Documents/GitHub/usgs-map-gwmodels
/Users/aleaf/Documents/GitHub/modflow-setup
/Users/aleaf/Documents/GitHub/modflow-export
/Users/aleaf/Documents/GitHub/modflow-obs
/Users/aleaf/Documents/GitHub/linesink-maker
/Users/aleaf/Documents/GitHub/gisutils
/Users/aleaf/Documents/GitHub/pydrograph
another_subfolder/scripts


### Using ``sys.path`` to import code from an arbitrary location

1) Using a text editor such as VSCode (or ``pathlib`` and python) make a new ``*.py`` file in another folder (anything in the same folder as this notebook can already be imported). For example:

In [78]:
subfolder = Path('another_subfolder/scripts')
subfolder.mkdir(exist_ok=True, parents=True)

with open(subfolder / 'mycode.py', 'w') as dest:
    dest.write("stuff = {'this is': 'a dictionary'}")

Now add this folder to the python path

In [80]:
sys.path.append('another_subfolder/scripts')

Code can be imported by calling the containing module

In [82]:
from mycode import stuff

stuff

{'this is': 'a dictionary'}

**Note**: Generally, importing code using ``sys.path`` is considered bad practice, because 

* it can hide dependencies.    

    * from the information above, we don't know whether ``mycode`` is a package that is installed, a module in the current folder, or anywhere else for that matter.
    * Similarly, we know that any modules from ``'another_subfolder/scripts'`` can be imported, but we don't know which modules in that folder are needed without some additional checking.

* importing code using ``sys.path`` is also sensitive to the location of the script relative to the path. If the script is moved or used on someone else's computer with a different file structure, it'll break.

In general, [installing reusable code in a package is the best way to go](https://nsls-ii.github.io/scientific-python-cookiecutter/). Packages provide a framework for organizing, documenting, testing and sharing code in a way that is easily understood by others.

Whatever you do, avoid importing with an `*` (i.e. ``from mycode import *``) at all costs. This imports everything from the namespace of a module, which can lead to unintended consequences.

## ``os`` — Miscellaneous operating system interfaces¶
Historically, the ``os.path`` module was the de facto standard for file and path manipulation. Since python 3.4 however, ``pathlib`` is generally cleaner and easier to use for most of these operations. But there are some exceptions.

### Changing the current working directory
``pathlib`` doesn't do this.   
Note: this can obviously lead to trouble in scripts, so should usually be avoided, but sometimes it is necessary.

In [84]:
# Example of changing the working directory
old_wd = os.getcwd()

# Go up one directory
os.chdir('..')
cwd = os.getcwd()
print ('Now in: ', cwd)

# Change back to original
os.chdir(old_wd)
cwd = os.getcwd()
print('Switched back to: ', cwd)

Now in:  /Users/aleaf/Documents/Python_course/python-for-hydrology/notebooks
Switched back to:  /Users/aleaf/Documents/Python_course/python-for-hydrology/notebooks/part0_python_intro


### os.walk

os.walk() is a great way to recursively generate all the file names and folders in a directory.  The following shows how it can be used to identify large directories.

In [110]:
pth = Path('..')
results = list(os.walk(pth))
results

[('..', ['part1_flopy', 'part0_python_intro'], ['.DS_Store']),
 ('../part1_flopy', ['data'], []),
 ('../part1_flopy/data', [], ['mt']),
 ('../part0_python_intro',
  ['another_subfolder', '.ipynb_checkpoints', 'more_files', 'data'],
  ['.DS_Store',
   '00_python_basics_review.ipynb',
   'test.py',
   'numpy.ipynb',
   '03_sys-os-shutil-pathlib.ipynb']),
 ('../part0_python_intro/another_subfolder', ['scripts'], []),
 ('../part0_python_intro/another_subfolder/scripts',
  ['__pycache__'],
  ['mycode.py']),
 ('../part0_python_intro/another_subfolder/scripts/__pycache__',
  [],
  ['mycode.cpython-310.pyc']),
 ('../part0_python_intro/.ipynb_checkpoints',
  [],
  ['03_sys-os-shutil-pathlib-checkpoint.ipynb']),
 ('../part0_python_intro/more_files', ['subfolder'], []),
 ('../part0_python_intro/more_files/subfolder', [], ['stuff.txt']),
 ('../part0_python_intro/data', [], ['mt'])]

#### Make a more readable list of files
Note: the key advantage of ``os.walk`` over ``glob`` is the recursion-- individual subfolder levels don't need to be known or specified a priori.

In [119]:
for root, dirs, files in os.walk(pth):
    for f in files:
        filepath = Path(root, f)
        print(filepath)

../.DS_Store
../part1_flopy/data/mt
../part0_python_intro/.DS_Store
../part0_python_intro/00_python_basics_review.ipynb
../part0_python_intro/test.py
../part0_python_intro/numpy.ipynb
../part0_python_intro/03_sys-os-shutil-pathlib.ipynb
../part0_python_intro/another_subfolder/scripts/mycode.py
../part0_python_intro/another_subfolder/scripts/__pycache__/mycode.cpython-310.pyc
../part0_python_intro/.ipynb_checkpoints/03_sys-os-shutil-pathlib-checkpoint.ipynb
../part0_python_intro/more_files/subfolder/stuff.txt
../part0_python_intro/data/mt


### Accessing environmental variables

In [142]:
os.environ

environ{'__CFBundleIdentifier': 'com.apple.Terminal',
        'TMPDIR': '/var/folders/4x/bmhyjcdn3mgfdvkk_jgz6bsrlnfk3t/T/',
        'XPC_FLAGS': '0x0',
        'LaunchInstanceID': 'A044076A-E805-4B47-8AD9-AD3E972138B9',
        'TERM': 'xterm-color',
        'SSH_AUTH_SOCK': '/private/tmp/com.apple.launchd.rEhJlhDi8n/Listeners',
        'SECURITYSESSIONID': '186aa',
        'XPC_SERVICE_NAME': '0',
        'TERM_PROGRAM': 'Apple_Terminal',
        'TERM_PROGRAM_VERSION': '440',
        'TERM_SESSION_ID': 'F3AC17EA-CF35-424E-8E65-DAFA84F0EA1B',
        'SHELL': '/bin/zsh',
        'HOME': '/Users/aleaf',
        'LOGNAME': 'aleaf',
        'USER': 'aleaf',
        'PATH': '/Users/aleaf/opt/anaconda3/envs/gis/bin:/Users/aleaf/opt/anaconda3/condabin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Users/aleaf/Documents/software/pestpp-binaries/mac:/Users/aleaf/Documents/software/modflow_exes',
        'SHLVL': '1',
        'PWD': '/Users/aleaf',
        'OLDPWD': '/Users/aleaf',
        'SS

#### Example: get the location of the current python (Conda) environment

In [143]:
os.environ['CONDA_PREFIX']

'/Users/aleaf/opt/anaconda3/envs/gis'

## ``subprocess`` — Subprocess management

The subprocess module offers a way to execute system commands, for example MODFLOW, or any operating system command that you can type at the command line.

The recommended approach to invoking subprocesses is to use the ``run()`` function for all use cases it can handle. For more advanced use cases, the underlying ``Popen`` interface can be used directly.

Take a look at the following help descriptions for ``run``.

Note, that on Windows, you may have to specify "shell=True" in order to access system commands.

In [133]:
help(subprocess.run)

Help on function run in module subprocess:

run(*popenargs, input=None, capture_output=False, timeout=None, check=False, **kwargs)
    Run command with arguments and return a CompletedProcess instance.
    
    The returned instance will have attributes args, returncode, stdout and
    stderr. By default, stdout and stderr are not captured, and those attributes
    will be None. Pass stdout=PIPE and/or stderr=PIPE in order to capture them.
    
    If check is True and the exit code was non-zero, it raises a
    CalledProcessError. The CalledProcessError object will have the return code
    in the returncode attribute, and output & stderr attributes if those streams
    were captured.
    
    If timeout is given, and the process takes too long, a TimeoutExpired
    exception will be raised.
    
    There is an optional argument "input", allowing you to
    pass bytes or a string to the subprocess's stdin.  If you use this argument
    you may not also use the Popen constructor's "std

In [136]:
# if on mac/unix
print(subprocess.run(['ls', '-l'], shell=True))

00_python_basics_review.ipynb
03_sys-os-shutil-pathlib.ipynb
11_xarray_mt_rainier_precip.ipynb
another_subfolder
data
junk.zip
more_files
numpy.ipynb
solutions
test.py
CompletedProcess(args=['ls', '-l'], returncode=0)


In [140]:
# if on windows
print(subprocess.run(['dir'], shell=True))
output

CompletedProcess(args=['dir'], returncode=127)


/bin/sh: dir: command not found


CompletedProcess(args=['dir'], returncode=127)

## ``zipfile`` — Work with ZIP archives

#### zip up one of the files in data/

In [129]:
with zipfile.ZipFile('junk.zip', 'w') as dest:
    dest.write('data/xarray/daymet_prcp_rainier_1980-2018.nc')

#### now extract it

In [130]:
with zipfile.ZipFile('junk.zip') as src:
    src.extract('data/xarray/daymet_prcp_rainier_1980-2018.nc', path='extracted_data')

## Testing Your Skills with a truly awful example:

#### the problem:
Pretend that the file `data/fileio/netcdf_data.zip` contains some climate data that we downloaded. If you open `data/fileio/netcdf_data.zip`, you'll see that within a subfolder `zipped` are a bunch of additional subfolders, each for a different year. Within each subfolder is another zipfile. Within each of these zipfiles is yet another subfolder, inside of which is the actual data file we want (`prcp.nc`). 

#### the goal:
To extract all of these `prcp.nc` files into a single folder, after renaming them with their respective years (obtained from their enclosing folders or zip files). e.g.  
```
prcp_1980.nc
prcp_1981.nc
...
```
This will allow us to open them together as a dataset in `xarray` (more on that later). Does this sound awful? I'm not making this up. This is the kind of structure you get if when downloading tiles of climate data with the [Daymet Tile Selection Tool](https://daymet.ornl.gov/gridded/)

#### hint:
you might find these functions helpful:
```
glob.glob
os.path.isdir
os.makedirs
zipfile.ZipFile
os.path.split
os.path.splitext
os.path.join
shutil.move
os.rename
os.rmdir
```

### solution

First, extract the master zipfile

In [29]:
with zipfile.ZipFile('data/fileio/netcdf_data.zip') as src:
    src.extractall('data/fileio/')

Make a list of the zipfiles

In [30]:
zipfiles = sorted(glob.glob('data/fileio/netcdf_data/zipped/*/*.zip'))
zipfiles[:5]

['data/fileio/netcdf_data/zipped/zipped_1980/12270_1980.zip',
 'data/fileio/netcdf_data/zipped/zipped_1981/12270_1981.zip',
 'data/fileio/netcdf_data/zipped/zipped_1982/12270_1982.zip',
 'data/fileio/netcdf_data/zipped/zipped_1983/12270_1983.zip',
 'data/fileio/netcdf_data/zipped/zipped_1984/12270_1984.zip']

In [31]:
# declare a destination path
dest_path = 'extracted_data'
variable = 'prcp'

for f in zipfiles:
    with zipfile.ZipFile(f) as src:
        # get the path to the source file and the year
        _, fname = os.path.split(f)
        name = os.path.splitext(fname)[0].replace('.tar', '')
        srcfile = '{}/{}.nc'.format(name, variable)
        year = name.split('_')[1]

        # where we want the extracted .nc file to end up
        destfile = os.path.join(dest_path, '{}_{}.nc'.format(variable, year))

        # extract the srcfile path to the /daymet folder
        # unfortunately this extracts the whole path, not just the file
        src.extract(srcfile, dest_path)
        # move the file up from subfolders to /daymet
        shutil.move(os.path.join(dest_path, srcfile), dest_path)
        # rename to include year
        os.rename(os.path.join(dest_path, '{}.nc'.format(variable)),
                  destfile)
        # trash subfolders that were extracted
        os.rmdir(os.path.join(dest_path, name))
        print('{}/{} --> {}'.format(f, srcfile, destfile))

data/fileio/netcdf_data/zipped/zipped_1980/12270_1980.zip/12270_1980/prcp.nc --> extracted_data/prcp_1980.nc
data/fileio/netcdf_data/zipped/zipped_1981/12270_1981.zip/12270_1981/prcp.nc --> extracted_data/prcp_1981.nc
data/fileio/netcdf_data/zipped/zipped_1982/12270_1982.zip/12270_1982/prcp.nc --> extracted_data/prcp_1982.nc
data/fileio/netcdf_data/zipped/zipped_1983/12270_1983.zip/12270_1983/prcp.nc --> extracted_data/prcp_1983.nc
data/fileio/netcdf_data/zipped/zipped_1984/12270_1984.zip/12270_1984/prcp.nc --> extracted_data/prcp_1984.nc
data/fileio/netcdf_data/zipped/zipped_1985/12270_1985.zip/12270_1985/prcp.nc --> extracted_data/prcp_1985.nc
data/fileio/netcdf_data/zipped/zipped_1986/12270_1986.zip/12270_1986/prcp.nc --> extracted_data/prcp_1986.nc
data/fileio/netcdf_data/zipped/zipped_1987/12270_1987.zip/12270_1987/prcp.nc --> extracted_data/prcp_1987.nc
data/fileio/netcdf_data/zipped/zipped_1988/12270_1988.zip/12270_1988/prcp.nc --> extracted_data/prcp_1988.nc
data/fileio/netcdf_

## Bonus -- Determining the location of an executable

There are often times that you run an executable that is nested somewhere deep within your system path.  It can often be a good idea to know exactly where that executable is located.  This might help you one day from accidently using an older version of an executable, such as MODFLOW.

In [32]:
# Define two functions to help determine 'which' program you are using
def is_exe(fpath):
    """
    Return True if fpath is an executable, otherwise return False
    """
    return os.path.isfile(fpath) and os.access(fpath, os.X_OK)

def which(program):
    """
    Locate the program and return its full path.  Return
    None if the program cannot be located.
    """
    fpath, fname = os.path.split(program)
    if fpath:
        if is_exe(program):
            return program
    else:
        # test for exe in current working directory
        if is_exe(program):
            return program
        # test for exe in path statement
        for path in os.environ["PATH"].split(os.pathsep):
            path = path.strip('"')
            exe_file = os.path.join(path, program)
            if is_exe(exe_file):
                return exe_file
    return None

In [33]:
which('MODFLOW-NWT_64.exe')